In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data()

In [4]:
x_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [5]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(('<pad>', '<sos>', '<unk>')):
    id_to_word[id_] = token
' '.join([id_to_word[id_] for id_ in x_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [6]:
import tensorflow_datasets as tfds

datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)

In [7]:
datasets.keys()

dict_keys(['train', 'test', 'unsupervised'])

In [8]:
train_size = info.splits['train'].num_examples
test_size = info.splits['test'].num_examples
train_size, test_size

(25000, 25000)

In [9]:
for x_batch, y_batch in datasets['train'].batch(5).take(1):
    for review, label in zip(x_batch.numpy(), y_batch.numpy()):
        print(f'Review: {review.decode("utf-8")[:200]} ...')
        print(f'Label: {label}', '= Positive' if label else '= Negative')
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative

Review: Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0 = Negative

Review: This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1 = Positive

Review: As others have mentioned, all the women that go nude in 

In [10]:
def preprocess(x_batch, y_batch):
    x_batch = tf.strings.substr(x_batch, 0, 300)
    x_batch = tf.strings.regex_replace(x_batch, rb'<br\s*/?>', b' ')
    x_batch = tf.strings.regex_replace(x_batch, b"[^a-zA-Z']", b' ')
    x_batch = tf.strings.split(x_batch)
    return x_batch.to_tensor(default_value=b'<pad>'), y_batch

In [11]:
preprocess(x_batch, y_batch)

(<tf.Tensor: shape=(5, 59), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
         b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'c

In [12]:
from collections import Counter

vocabulary = Counter()
for x_batch, y_batch in datasets['train'].batch(32).map(preprocess):
    for review in x_batch:
        vocabulary.update(list(review.numpy()))

In [13]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [14]:
len(vocabulary)

53893

In [15]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [16]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b'This movie was faaaantastic'.split():
    print(word_to_id.get(word) or vocab_size)

22
12
11
10000


In [17]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [18]:
table.lookup(tf.constant([b'This movie was faaaantastic'.split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10771]], dtype=int64)>

In [19]:
def encode_words(x_batch, y_batch):
    return table.lookup(x_batch), y_batch

In [20]:
train_set = datasets['train'].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [21]:
for x_batch, y_batch in train_set.take(1):
    print(x_batch)
    print(y_batch)

tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


In [22]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 16s 15ms/step - loss: 0.6560 - acc: 0.5852
Epoch 2/5
782/782 [==============================] - 11s 14ms/step - loss: 0.4482 - acc: 0.7914
Epoch 3/5
782/782 [==============================] - 11s 14ms/step - loss: 0.3206 - acc: 0.8666
Epoch 4/5
782/782 [==============================] - 12s 15ms/step - loss: 0.2098 - acc: 0.9217
Epoch 5/5
782/782 [==============================] - 12s 15ms/step - loss: 0.1451 - acc: 0.9498


In [23]:
# using masking
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, 
                           embed_size, input_shape=[None], 
                           mask_zero=True),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 22s 22ms/step - loss: 0.5374 - acc: 0.7186
Epoch 2/5
782/782 [==============================] - 17s 22ms/step - loss: 0.3425 - acc: 0.8575
Epoch 3/5
782/782 [==============================] - 17s 22ms/step - loss: 0.1970 - acc: 0.9296
Epoch 4/5
782/782 [==============================] - 17s 22ms/step - loss: 0.1439 - acc: 0.9489
Epoch 5/5
782/782 [==============================] - 17s 22ms/step - loss: 0.1112 - acc: 0.9596


In [24]:
# using manual masking

K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation='sigmoid')(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 21s 22ms/step - loss: 0.5336 - acc: 0.7244
Epoch 2/5
782/782 [==============================] - 17s 22ms/step - loss: 0.3454 - acc: 0.8572
Epoch 3/5
782/782 [==============================] - 17s 22ms/step - loss: 0.1980 - acc: 0.9288
Epoch 4/5
782/782 [==============================] - 17s 22ms/step - loss: 0.1473 - acc: 0.9468
Epoch 5/5
782/782 [==============================] - 17s 22ms/step - loss: 0.1174 - acc: 0.9565
